In [ ]:
# Import the necessary libraries 
import pandas as pd
from matplotlib import pyplot as plt
from reservoirpy.nodes import Reservoir, Ridge, Input
import numpy as np
from reservoirpy.observables import nrmse, rsquare
import json
from reservoirpy.hyper import research
from reservoirpy.hyper import plot_hyperopt_report
from reservoirpy.observables import rmse, rsquare, nrmse

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [ ]:
emg_data_train = pd.read_csv('./Kinematics_Train_Data_EMG.csv')
emg_data_test = pd.read_csv('./Kinematics_Test_Data_EMG.csv')
angles_data_train = pd.read_csv('./Kinematics_Train_Data_Angles.csv')
angles_data_test = pd.read_csv('./Kinematics_Test_Data_Angles.csv')

In [ ]:
#angles_data_train = angles_data_train.drop(columns = ["Lead_Pelvis_Ant", "Lead_Pelvis_Olb", "Lead_Pelvis_IE","Trail_Pelvis_Ant", "Trail_Pelvis_Olb", "Trail_Pelvis_IE"])
#angles_data_test = angles_data_test.drop(columns = ["Lead_Pelvis_Ant", "Lead_Pelvis_Olb", "Lead_Pelvis_IE","Trail_Pelvis_Ant", "Trail_Pelvis_Olb", "Trail_Pelvis_IE"])
angles_data_test
len(emg_data_train)
emg_data_train_test = np.array(emg_data_train)
data_per_subject = 101
n_subjects = int (emg_data_train_test.shape[0]/data_per_subject)

emg_data_train_test = emg_data_train_test.reshape(n_subjects, data_per_subject, 16)

angles_data_train_test = np.array(angles_data_train)
angles_data_train_test = angles_data_train_test.reshape(n_subjects, data_per_subject, 20)

# how many output angles you want to predict
n_angles = 17
angles_data_train_test = angles_data_train_test[:, :, 0:n_angles]

emg_data_train_test_ev = np.array(emg_data_train)
emg_data_train_test_ev

emg_data_test_test = np.array(emg_data_test)
emg_data_test_test = emg_data_test_test.reshape(n_subjects, data_per_subject, 16)

#emg_data_test_test = emg_data_test_test.reshape(14, data_per_subject, 16)
emg_data_test_test

angles_data_test_test = np.array(angles_data_test)
#angles_data_test_test = emg_data_test_test.reshape(14, data_per_subject, 16)
angles_data_test_test = angles_data_test_test.reshape(n_subjects, data_per_subject, 20)
angles_data_test_test = angles_data_test_test[:, :, 0:n_angles]

# emg_left_01_test = np.array(emg_left_01)
# emg_left_01_test = emg_left_01_test.reshape(1, data_per_subject, 16)
# emg_left_01_test
# angles_left_01_test = np.array(angles_left_01)
# angles_left_01_test = angles_left_01_test.reshape(1, data_per_subject, 20)
# angles_left_01_test

x_train = emg_data_train_test
y_train = angles_data_train_test
x_test = emg_data_test_test
y_test = angles_data_test_test
x_train_ev = emg_data_train_test_ev

In [ ]:
print (x_train.shape)
print (y_train.shape)
print (x_test.shape)
print (y_test.shape)
print (x_train_ev.shape)

In [ ]:
batch_size = 1
# Each MNIST image batch is a tensor of shape (batch_size, 28, 28).
# Each input sequence will be of size (28, 28) (height is treated like time).
input_dim = (data_per_subject, 16)

units = 512
output_size = 101  # labels are from 0 to 9

# Build the RNN model
def build_model(allow_cudnn_kernel=True):
    # CuDNN is only available at the layer level, and not at the cell level.
    # This means `LSTM(units)` will use the CuDNN kernel,
    # while RNN(LSTMCell(units)) will run on non-CuDNN kernel.
    model = keras.models.Sequential(
        [
            #keras.layers.GRU(256, return_sequences=True),
            #keras.layers.RNN(keras.layers.LSTMCell(units)),
            keras.layers.SimpleRNN(units, input_shape=( 101, 16)),
            #keras.layers.Embedding(input_dim=101, output_dim=int((output_size*n_angles)/units)),
            keras.layers.Embedding(input_dim=101, output_dim=8),
            keras.layers.BatchNormalization(),
            tf.keras.layers.Conv1D(n_angles, units-(data_per_subject-1)),
            keras.layers.Dense(n_angles),
            #keras.layers.Reshape((101, n_angles), input_shape=(101*n_angles,)),
        ]
    )
    return model

model = build_model()
model.summary()

In [ ]:
model = build_model(allow_cudnn_kernel=True)

model.compile(
    loss=keras.losses.MeanAbsoluteError(),
    optimizer="sgd",
    metrics=["accuracy"],
)


model.fit(
    x_train, y_train, batch_size=batch_size, epochs=200
)


In [ ]:
mse = tf.keras.losses.MeanSquaredError()

mse_train = mse (model.predict(x_train), y_train)
mse_test = mse (model.predict(x_test), y_test)

# root mean squared error
rmse_train = mse_train**0.5
rmse_test = mse_test**0.5

print ("training MSE", mse_train)
print ("test MSE", mse_test)

print ("training RMSE", rmse_train)
print ("test RMSE", rmse_test)

In [ ]:
model.predict(x_train).shape

In [ ]:
y_train.shape

In [ ]:
import numpy as np
import scipy.stats as stats

Array1 = y_train
Array2 = model.predict_on_batch(x_train)
resultArray = np.ones(n_subjects*data_per_subject*n_angles).reshape(n_subjects,data_per_subject,n_angles)
for i in range(Array1.shape[0]):
        for j in range(Array1.shape[1]):
            for k in range(Array1.shape[2]):
                slope, intercept, r_value, p_value, std_err = stats.linregress(
                    Array1[i, j, :],
                    Array2[i, j, :])
                R2 = r_value**2
                resultArray[ i , j , k] = R2

np.mean(resultArray)

In [ ]:
resultArray.shape